# 악플 다중 분류 - KoBERT

## Colab 환경 설정

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 190 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 8.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595733 sha256=4d1cbd1c2ff8ef4b7ffc6170c717210b4d4772042ee11c3a3dea28728bfe5fb3
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 8.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-l9rp0ox8
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-l9rp0ox8
     |████████████████████████████████| 132 kB 5.9 MB/s 
     |████████████████████████████████| 4.5 MB 55.4 MB/s 
     |████████████████████████████████| 4.2 MB 64.2 MB/s 
     |████████████████████████████████| 86 kB 3.5 MB/s 
     |████████████████████████████████| 596 kB 53.8 MB/s 
     |████████████████████████████████| 6.6 MB 31.1 MB/s 
     |████████████████████████████████| 79 kB 9.7 MB/s 
     |████████████████████████████████| 8.9 MB 46.0 MB/s 
     |████████████████████████████████| 127 kB 75.6 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=a9a502c1a31d254a1597bfe41b5e95bec19fa4b962731bbc1ef3f66fc9875517
  Stored in directory: 

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## 데이터 전처리

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/content/common_data1.csv")

In [ ]:
# 분류가 안되어 있는 데이터 확인
print(df[df.sum(axis=1) == 0].index)

# 분류 안되어 있는 데이터 삭제
df = df[df.sum(axis=1) != 0]

Int64Index([38299, 44356], dtype='int64')


In [ ]:
train_dataset = pd.DataFrame()
for i in range(1, 12):
  조건 = df.iloc[:, i] > 0
  data = df[조건][df.columns[i]] * (i-1)
  data = pd.concat([df[조건]['문장'], data], axis=1)
  data.columns = ['문장', '라벨']
  train_dataset = pd.concat([train_dataset, data])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, _, _ = train_test_split(train_dataset, train_dataset['라벨'], test_size=0.2)

## 텍스트 전처리

In [ ]:
train_data = []
for q, label in zip(X_train['문장'], X_train['라벨'])  :
    data = []
    data.append(q)
    data.append((label))
    train_data.append(data)

In [ ]:
print(train_data[0])
print(train_data[10])
print(train_data[500])

['여자라고 치고 저자리에서 따먹어도 되면 인정한다', 0.0]
['역사 역사 그렇게 역사중요하게여기는분들이 정작 살인마 짱깨들 변호하던새끼를 대통령으로앉힘ㅋㅋ', 3.0]
['지들끼리 ㅆ 욕을하던 왜 남들이 더 난리야  집안싸움 아닌가    욕하는 늬들이 언제 애네들 걱정했다고    나도 포기한 놈들인데 늬들이 뭔데     박근혜탄핵이면 김무성같은넘은 아직도 면상 디밀면 안되지  김문수 욕 바가지먹을 발언인건 본인도 알터  틀린소리는 아니지  조국이나 까라고     지나가는 말꼬리잡고 쌩난리 치지말고', 10.0]


In [ ]:
test_data = []
for q, label in zip(X_test['문장'], X_test['라벨'])  :
    data = []
    data.append(q)
    data.append((label))

    test_data.append(data)

In [ ]:
print(test_data[0])
print(test_data[10])
print(test_data[500])

['드라이브 스루는 위에 댓글처럼 미국이 원조인데    대체 왜 팩트를 왜곡하지  그리 반일이 좋아  기레기야', 7.0]
['원 리더십  원 메시지로 내부 결속을 더 강화하고 외부로 리마커블            한 이미지를 구축하여 다시 교회로 모일수 있기를 소망하고 기도합니다', 9.0]
['김지현                 문병기 기레기들아  뭔 기사를 이 따위로 쓰냐   시종일관 야당 얘기만 잔뜩 해놓고  마지막에 한다는 말이 고작 여권 관계자도 청와대를 비판했다라고 씨부려놓았는데  그런 지적을 한 여당 인사의 이름을 밝히지 않는 한  니들 기레기들이 적당히 거짓말하고 있는 것인지 어떻게 판단하란 말이냐  이런 걸 기사라고 내놓는 똥아일보는 정말 한심하다  발언한 자들의 이름을 못 밝힐 바에는 기사를 쓰지를 말든가', 7.0]


## KoBERT 입력 데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 70
batch_size = 128
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[0]

(array([   2, 3318, 6004,  517, 7484, 3990, 7159, 6903, 1833, 6183, 6855,
        5859, 1766, 3785, 7831,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1], dtype=int32),
 array(16, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], dtype=int32),
 0)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## KoBERT 학습 모델 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

## KoBERT 모델 학습시키기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/376 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.450798511505127 train acc 0.109375
epoch 1 batch id 201 loss 1.5767631530761719 train acc 0.2457245024875622
epoch 1 train acc 0.4199426529255319


  0%|          | 0/94 [00:00<?, ?it/s]

epoch 1 test acc 0.650157272913257


  0%|          | 0/376 [00:00<?, ?it/s]

...

epoch 9 test acc 0.6274038461538461


  0%|          | 0/376 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.43986549973487854 train acc 0.7890625
epoch 10 batch id 201 loss 0.485699862241745 train acc 0.7886738184079602
epoch 10 train acc 0.7914311835106383


  0%|          | 0/94 [00:00<?, ?it/s]

epoch 10 test acc 0.6224235372340425
